In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')

In [3]:
df.sample(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
268,0.54050,20.0,3.97,0,0.575,7.470,52.6,2.8720,5,264,13.0,390.30,3.16,43.5
413,28.65580,0.0,18.10,0,0.597,5.155,100.0,1.5894,24,666,20.2,210.97,20.08,16.3
246,0.33983,22.0,5.86,0,0.431,6.108,34.9,8.0555,7,330,19.1,390.18,9.16,24.3


In [7]:
df.shape

(506, 14)

In [8]:
df.isnull().mean()*100

crim       0.0
zn         0.0
indus      0.0
chas       0.0
nox        0.0
rm         0.0
age        0.0
dis        0.0
rad        0.0
tax        0.0
ptratio    0.0
b          0.0
lstat      0.0
medv       0.0
dtype: float64

In [6]:
X=df.drop(columns=['medv'])
Y=df['medv']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)
X_train.shape,X_test.shape

((404, 13), (102, 13))

In [14]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()

Cross Validation (Kfold)

In [17]:
from sklearn.model_selection import cross_val_score,KFold

kfold=KFold(n_splits=5,shuffle=True,random_state=10)
score=cross_val_score(knn,X_train,Y_train,scoring='r2',cv=kfold)
score.mean()

0.5019816036989816

Grid Search CV

In [18]:
from sklearn.model_selection import GridSearchCV
params={
    "weights":['uniform', 'distance'],
    "algorithm":['auto','ball_tree','kd_tree','brute'],
    "n_neighbors":[5,6,7,8,9,10,12],
    "p":[1,2]   
}

In [ ]:
knn=KNeighborsRegressor()

In [19]:
grid=GridSearchCV(knn,params,cv=kfold,scoring='r2',refit=True)

In [20]:
grid.fit(X_train,Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=10, shuffle=True),
             estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [5, 6, 7, 8, 9, 10, 12], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='r2')

In [21]:
grid.best_params_

{'algorithm': 'auto', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [23]:
grid.best_score_

0.6134069444453479

In [28]:
pd.DataFrame(grid.cv_results_)[["param_algorithm","param_n_neighbors","param_p","param_weights","mean_test_score"]].sort_values("mean_test_score",ascending=False)

,param_algorithm,param_n_neighbors,param_p,param_weights,mean_test_score
29,ball_tree,5,1,distance,0.613407
1,auto,5,1,distance,0.613407
57,kd_tree,5,1,distance,0.613407
85,brute,5,1,distance,0.613407
37,ball_tree,7,1,distance,0.604528
...,...,...,...,...,...
46,ball_tree,9,2,uniform,0.444791
82,kd_tree,12,2,uniform,0.439904
26,auto,12,2,uniform,0.439904
110,brute,12,2,uniform,0.439904


In [30]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
rsc=RandomizedSearchCV(knn,params,scoring='r2',cv=kfold,refit=True)
rsc.fit(X_train,Y_train)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=10, shuffle=True),
                   estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'n_neighbors': [5, 6, 7, 8, 9, 10, 12],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   scoring='r2')

In [34]:
rsc.best_params_

{'weights': 'distance', 'p': 1, 'n_neighbors': 8, 'algorithm': 'brute'}

In [35]:
rsc.best_score_

0.5988284923787147

**Important Points**

1. The two most commonly use hyperparameter tuning techniques are GridSearchCV and RandomSearchCV.
2. GridSearchCV: It is an exhaustive search/brute force solution to the idea of finding the best combination of parameters.As the name suggests it makes Grid and does cross validation for each combination,  and finds the best set of combination. Though, it sounds as an convincing appraoch, in data science the dimensions of the dataset can go to a large number. Take Gradient Boosting as an example, it can take 30 hyperparemters and say each hyperparameter takes 5 values, then it'll be 30 dimensional and will have to search through 150 different combination excluding the cross validation. This can be tedious and can take a lot of time and memory. But given you use Grid Search CV, it'll give you with the best combinations of hyperparameters.
3. Random Search CV: This technique relies on luck, since unlike Grid Search CV it does not consider all the combinations and only selects k (choosen by the data engineer) combinations. Thus, random search CV will mostly not give you the best combinations. But still, it's better when the dataset has a lot of features.
You can use GridSearchCV only when your search space is small, or after RandomSearchCV narrowed it down.